In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import base64
import json
import logging
import os
import re
import smtplib
import sys
import time
import threading
from copy import deepcopy
from functools import partial
from itertools import chain, combinations, product
from numbers import Number
from pathlib import Path
from uuid import uuid4
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser
from scipy.optimize import minimize

from api.email import smtp_server, to_email_test, to_email_all, df_to_html, from_email
from api.risk.var import VaRPnl, get_db_from_clarion_positions, get_fut_trade_spec, get_var_ts, get_db_from_specs, var_ptf, get_clarion_fut_trade_spec
from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods, parse_swap_period
from gioutils.tools_clarion import get_clarion_positions
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
    SQL
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root,
    encode_df_as_image_tag
)

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)
rng = np.random.default_rng(7)

In [4]:
super_clarion(clarion)
runner = Runner(clarion)

In [5]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-05-12 12:16:40,614 - Authenticating...
2023-05-12 12:16:41,299 - Successfully authenticated


### METRICS

In [6]:
clarion_metrics = dict(
    pv = clarion.metric('pv', 'PresentValue', 'Any'),
    local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any'),
    # currency = clarion.metric('currency', 'PayCurrency', 'Any'),
    strike = clarion.metric('strike', 'Strike', 'Any'),
    notional = clarion.metric('notional', 'Notional', 'Any'),
    expiry = clarion.metric('expiry', 'ExpiryDate', 'Any'),
    maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'}),
    start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'}),
    fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates'),
    delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    gamma = clarion.metric('gamma', 'Gamma', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'}),
    price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'}),
    fwd_price_cents = clarion.metric('fwd_price_cents', 'ForwardPrice', 'Rates', {'Unit': 'Cents'}),
    price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'}),
    price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'}),
    price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'}),
    spot = clarion.metric('spot', 'Spot', 'Rates'),
    # vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'}),
    # atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'}),
    atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'}),
    # strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'}),
    strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'}),
    # carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'}),
)

### PRICER

In [9]:
trades = [
    clarion.instruments.RatesShortTermFuture(
        currency='USD',
        instrument_name='FEDFUNDS AVG 1M CME',
        tenor='Q3',
        notional=1,
    ),

    clarion.instruments.RatesShortTermFuture(
        currency='USD',
        instrument_name='SOFR 3M CME',
        tenor='Z3',
        notional=1,
    ),
            
    clarion.instruments.RatesSwaption(
        currency='EUR',
        instrument_name='EURIBOR A 3M',
        expiry='9m',
        tenor='1y',
        strike='3.65',
        swaption_type='Pay',
        notional=2e9,
        settle_type='Cash Price',
        collateral_currency='USD',
        pay_settle='Spot',
        # id='eur_9m_bear_flat'
    ),
           
    clarion.instruments.RatesSwap(
        currency='EUR',
        instrument_name='ESTER A',
        notional=261e6,
        strike='a',
        start_date='2026-01-13',
        tenor='2y',
        swap_type='Pay',
        clearing_house='LCH',
    ),
    clarion.instruments.RatesCmsSpreadSingleLook(
        currency='USD',
        instrument_name='SOFR CMS 10Y-2Y',
        start_date='1y',
        notional=-1.6e9,
        strike='a + 124',
        cap_floor_type='Cap',
        collateral_currency='USD',
        pay_settle='Spot',
    ),
]

In [19]:
clarion.instruments

metricType assetClass  \
risk_metrics      0        IntrinsicValue        Any   
                  1     LocalPresentValue        Any   
                  2           OptionTheta        Any   
                  3           OptionTheta        Any   
                  4           OptionTheta        Any   
...                                   ...        ...   
positions_metrics 93           Volatility      Rates   
                  94           Volatility      Rates   
                  95  YieldYieldAssetSwap      Rates   
                  96              ZSpread      Rates   
                  97              ZSpread      Rates   

                                                    config  
risk_metrics      0                                         
                  1                                         
                  2                        {'Period': '1'}  
                  3                        {'Period': '7'}  
                  4                       {'Period': '30'}  
...                                                    ...  
positions_metrics 93  {'Period': 'Annual', 'Type': 'Skew'}  
                  94   {'Period': 'Daily', 'Type': 'Skew'}  
                  95                                        
                  96                   {'Type': 'Forward'}  
                  97                      {'Type': 'Spot'}  

[290 rows x 3 columns]

In [13]:
runner.run([*clarion_metrics.values()], trades)

2023-05-12 12:20:14,452 - Requesting data
2023-05-12 12:20:22,051 - Request complete
2023-05-12 12:20:22,226 - Response Count Summary - DFRAME:5, IS_ERROR:30


id periodType  \
0  ratesshorttermfuture_508845affef1430dbc09ffa1f...      daily   
1  ratesshorttermfuture_b6e7a7fc78e94d55b446a2380...      daily   
2     ratesswaption_ce6d83e0169440ca989b546565c3dbb5      daily   
3         ratesswap_b3d165b934434f7284ae399f3f3b4243      daily   
4  ratescmsspreadsinglelook_e467432c14144aabb43fa...      daily   

                                         description     period atm_vol_ann  \
0                         USD FEDFUNDS AVG 1M CME Q3 2023-05-12               
1                                 USD SOFR 3M CME Z3 2023-05-12               
2  EUR EURIBOR A 3M 9m 1y 3.65 Payer USD CashPric... 2023-05-12    111.3498   
3                           EUR ESTER A 13-Jan-26 2y 2023-05-12               
4        USD SOFR CMS 10Y-2Y 1y a + 124 Cap USD Spot 2023-05-12    101.3498   

     delta      expiry fwd_price_cents  fwd_rate    gamma  ...  _expiry  \
0 -41.1538                               95.0175   0.0010  ...      NaN   
1 -25.0000                               95.7525        0  ...      NaN   
2   59.67k  2024-02-12         17.1415    3.1257 930.2576  ...       9m   
3   48.09k                                2.3842 -46.6767  ...      NaN   
4 109.9824  2024-05-13          7.1592    0.1784  -2.8762  ...      NaN   

   _strike _swaption_type _collateral_currency  _settle_type  _pay_settle  \
0      NaN            NaN                  NaN           NaN          NaN   
1      NaN            NaN                  NaN           NaN          NaN   
2     3.65            Pay                  USD    Cash Price         Spot   
3        a            NaN                  NaN           NaN          NaN   
4  a + 124            NaN                  USD           NaN         Spot   

  _swap_type _start_date  _clearing_house _cap_floor_type  
0        NaN         NaN              NaN             NaN  
1        NaN         NaN              NaN             NaN  
2        NaN         NaN              NaN             NaN  
3        Pay  2026-01-13              LCH             NaN  
4        NaN          1y              NaN             Cap  

[5 rows x 36 columns]

### LIVE BOOK

In [14]:
df = get_clarion_positions(clarion, book='MU-BAL')

2023-05-12 12:21:29,262 - Requesting data
2023-05-12 12:21:46,434 - Request complete
2023-05-12 12:21:46,624 - Response Count Summary - DFRAME:395


### VAR

In [7]:
book = 'MU'
capital = 200e6
flag = 1.5 / 100 * capital

In [8]:
dv = VaRPnl()
wv = VaRPnl()
wv.get_data(
    offset="5y",
    exclude_cash=False,
    horizon='5d',
    book=book
)

C:\dev\projects\menashe\api\risk\var.py:172: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ).sum()


In [20]:
pd.Timestamp('2034-01-01')

Timestamp('2034-01-01 00:00:00')

In [21]:
ez

<module 'gioutils.ezutils' from 'C:\\dev\\projects\\gioutils\\gioutils\\ezutils.py'>

In [24]:
ez.date_to_xl('2024-07-01')

45474